In [2]:
# %% 0) Setup (minimal)
import duckdb, pandas as pd
def show(df, n=10): print(df.head(n).to_string(index=False))

# %% 1) Self-join (LC-181)
emp = pd.DataFrame({
  "id":[1,2,3,4], "name":["Ann","Bob","Cai","Dia"],
  "manager_id":[None,1,1,2], "salary":[100,120,90,130]
})
show(emp)

res_181 = duckdb.sql("""
SELECT w.name AS employee, m.name AS manager, w.salary AS emp_sal, m.salary AS mgr_sal
FROM emp w
JOIN emp m ON w.manager_id = m.id
WHERE w.salary > m.salary
""").df()
show(res_181)

# Quick checks
assert duckdb.sql("""
SELECT COUNT(*) FROM emp WHERE manager_id IS NOT NULL AND id = manager_id
""").fetchone()[0] == 0, "No one should manage themselves"
assert (res_181["emp_sal"] > res_181["mgr_sal"]).all(), "All rows must have emp_sal > mgr_sal"

# %% 2) Top-N per group (LC-184 — ROW_NUMBER())
dept = pd.DataFrame({
  "emp":["Ann","Bob","Cai","Dia","Eli","Flo"],
  "dept":["A","A","A","B","B","B"],
  "salary":[100,120,120,130,130,110]
})
show(dept)

top_per_dept = duckdb.sql("""
WITH ranked AS (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY dept ORDER BY salary DESC, emp) rn
  FROM dept
)
SELECT dept, emp, salary FROM ranked WHERE rn=1 ORDER BY dept
""").df()
show(top_per_dept)

# Check vs MAX(salary) per dept (pure Pandas, no temp view)
mx = dept.groupby("dept", as_index=False)["salary"].max().rename(columns={"salary":"m"})
assert top_per_dept.merge(mx, on="dept").eval("salary==m").all(), "Top row must match per-dept MAX salary"

# %% 3) Gaps & Islands (≥3 consecutive hot days) — no lateral join
stadium = pd.DataFrame({
 "id":[1,2,3,4,5,6,7],
 "visit_date":pd.to_datetime(["2023-01-01","2023-01-02","2023-01-03",
                              "2023-01-04","2023-01-05","2023-01-06","2023-01-07"]),
 "people":[10,120,130,140,50,160,170]
})
show(stadium)

streak3 = duckdb.sql("""
WITH mark AS (
  SELECT id, visit_date, people, (people>=100) AS hot
  FROM stadium
),
hot_rows AS (
  SELECT *,
         id - ROW_NUMBER() OVER (ORDER BY id) AS grp   -- same constant within a run
  FROM mark
  WHERE hot
),
kept AS (
  SELECT grp
  FROM hot_rows
  GROUP BY grp
  HAVING COUNT(*) >= 3
)
SELECT s.*
FROM hot_rows h
JOIN kept k USING (grp)
JOIN stadium s ON s.id = h.id
ORDER BY s.id
""").df()
show(streak3)

# Check every included hot island has length >= 3
bad_groups = duckdb.sql("""
WITH mark AS (SELECT id, (people>=100) AS hot FROM stadium),
hot_rows AS (
  SELECT *, id - ROW_NUMBER() OVER (ORDER BY id) AS grp
  FROM mark WHERE hot
),
kept AS (
  SELECT grp FROM hot_rows GROUP BY grp HAVING COUNT(*) >= 3
),
lens AS (SELECT grp, COUNT(*) AS n FROM hot_rows GROUP BY grp)
SELECT COUNT(*) 
FROM lens
JOIN kept USING (grp)
WHERE n < 3
""").fetchone()[0]
assert bad_groups == 0, "All included hot runs must be length ≥ 3"



# %% 4) Conditional aggregation + multi-join (LC-262)
trips = pd.DataFrame({
 "id":[1,2,3,4],
 "client_id":[10,11,10,12],
 "driver_id":[20,21,22,21],
 "status":["completed","cancelled_by_client","completed","cancelled_by_driver"],
 "request_at":pd.to_datetime(["2013-10-01","2013-10-01","2013-10-02","2013-10-03"])
})
users = pd.DataFrame({
 "users_id":[10,11,12,20,21,22],
 "role":["client","client","client","driver","driver","driver"],
 "banned":["No","No","No","No","No","No"]
})

cancel_rates = duckdb.sql("""
WITH j AS (
  SELECT DATE_TRUNC('day', t.request_at)::DATE AS day, t.status
  FROM trips t
  JOIN users c ON t.client_id=c.users_id AND c.role='client' AND c.banned='No'
  JOIN users d ON t.driver_id=d.users_id AND d.role='driver' AND d.banned='No'
)
SELECT day,
       ROUND( SUM((status!='completed')::INT)::DOUBLE / COUNT(*), 2) AS cancellation_rate
FROM j
GROUP BY day ORDER BY day
""").df()
show(cancel_rates)

# Checks
assert ((cancel_rates["cancellation_rate"]>=0) & (cancel_rates["cancellation_rate"]<=1)).all(), "Rates must be in [0,1]"
assert duckdb.sql("""
SELECT COUNT(*) FROM trips t
JOIN users c ON t.client_id=c.users_id
WHERE c.role='client' AND c.banned<>'No'
""").fetchone()[0] == 0, "No banned clients included"

# %% 5) Group median three ways (LC-569)
pay = pd.DataFrame({
 "company":["A","A","A","B","B","C"],
 "salary":[451,513,700,234,1154,2645]
})

m1 = duckdb.sql("SELECT company, median(salary) AS med FROM pay GROUP BY company ORDER BY company").df()
m2 = duckdb.sql("SELECT company, quantile_cont(salary,0.5) AS med FROM pay GROUP BY company ORDER BY company").df()
m3 = duckdb.sql("""
WITH r AS (
  SELECT *, COUNT(*) OVER (PARTITION BY company) n,
         ROW_NUMBER() OVER (PARTITION BY company ORDER BY salary) rn
  FROM pay
)
SELECT company, AVG(salary) AS med
FROM r
WHERE rn IN ((n+1)/2, (n+2)/2)
GROUP BY company
ORDER BY company
""").df()

# Float-safe equality checks
assert m1["med"].round(6).equals(m2["med"].round(6)), "median == quantile_cont"
merged_13 = m1.merge(m3, on="company", suffixes=("_1","_3"))
assert (merged_13["med_1"].round(6) == merged_13["med_3"].round(6)).all(), "median == windowed"

# %% 6) Pure-Pandas mirror
dept["rn"] = (
  dept.sort_values(["dept","salary"], ascending=[True,False])
      .groupby("dept").cumcount()+1
)
top_pd = dept.query("rn==1")[["dept","emp","salary"]].sort_values("dept")
show(top_pd)

maxs = dept.groupby("dept", as_index=False)["salary"].max().rename(columns={"salary":"m"})
pd_check = top_pd.merge(maxs, on="dept")
assert (pd_check["salary"] == pd_check["m"]).all(), "Pandas top-1 matches group MAX"


 id name  manager_id  salary
  1  Ann         NaN     100
  2  Bob         1.0     120
  3  Cai         1.0      90
  4  Dia         2.0     130
employee manager  emp_sal  mgr_sal
     Bob     Ann      120      100
     Dia     Bob      130      120
emp dept  salary
Ann    A     100
Bob    A     120
Cai    A     120
Dia    B     130
Eli    B     130
Flo    B     110
dept emp  salary
   A Bob     120
   B Dia     130
 id visit_date  people
  1 2023-01-01      10
  2 2023-01-02     120
  3 2023-01-03     130
  4 2023-01-04     140
  5 2023-01-05      50
  6 2023-01-06     160
  7 2023-01-07     170
 id visit_date  people
  2 2023-01-02     120
  3 2023-01-03     130
  4 2023-01-04     140
       day  cancellation_rate
2013-10-01                0.5
2013-10-02                0.0
2013-10-03                1.0


AssertionError: median == windowed